In [14]:
import torchaudio
import librosa
import torch
from df import enhance, init_df
import soundfile as sf
import torchaudio.functional as F
import os
import glob

In [11]:
def denoising_deepfilternet(audio_file_path, output_file_path, sample_rate=16000, atten_lim_db=-30, cutoff_freq=200, gain_db=8):
    # Load DeepFilterNet model
    model, df_state, _ = init_df()  # Ensure the model is correctly loaded

    # Load the audio file using librosa
    noisy_audio, sr = librosa.load(audio_file_path, sr=sample_rate)  # Resample to 16kHz
    noisy_audio = librosa.util.normalize(noisy_audio)  # Normalize amplitude
    noisy_audio, _ = librosa.effects.trim(noisy_audio)  # Trim silence

    # Convert to PyTorch tensor and ensure the correct shape
    noisy_audio_tensor = torch.tensor(noisy_audio, dtype=torch.float32).unsqueeze(0)  # Shape: (1, samples)

    # Ensure model is in evaluation mode
    model.eval()

    # Apply DeepFilterNet Enhancement with stronger noise suppression
    enhanced_audio = enhance(model, df_state, noisy_audio_tensor, atten_lim_db=atten_lim_db)  # Stronger filtering

    # Apply a high-pass filter to remove low-frequency noise
    enhanced_audio = torchaudio.functional.highpass_biquad(enhanced_audio, sr, cutoff_freq=cutoff_freq)  # Removes rumbling noise

    # Apply post-processing: Boost speech frequencies if needed
    enhanced_audio = F.gain(enhanced_audio, gain_db=gain_db)  # Boost speech clarity

    # Save the enhanced audio as WAV
    sf.write(output_file_path, enhanced_audio.squeeze().cpu().numpy(), sr)

    print("✅ Enhanced Audio Saved at given output_file_path")

In [12]:
# Default value: 16000 (16kHz)
SAMPLE_RATE = 16000

# Stronger filtering | recommended values: 1e-5 , -20 | Lower: More noticeable denoising
ATTEN_LIM_DB = -40

# Removes rumbling noise | Default value: 200
CUTOFF_FREQ = 200

# Helps restore voice clarity | Greater: Boosts speech frequencies | recommended values: 3, 5, 8, 10
GAIN_DB = 12

In [13]:
audio_file_path = r"C:\Users\User_1\Desktop\Comm-sounds\sat.mp3"
output_file_path = r"C:\Users\User_1\Desktop\Comm-sounds\output_1.wav"

denoising_deepfilternet(audio_file_path, output_file_path, sample_rate=SAMPLE_RATE, atten_lim_db=ATTEN_LIM_DB, cutoff_freq=CUTOFF_FREQ, gain_db=GAIN_DB)

2025-03-06 09:50:34 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-06 09:50:34 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-06 09:50:34 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-06 09:50:34 | INFO     | DF | Found checkpoint C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3\checkpoints\model_120.ckpt.best with epoch 120
2025-03-06 09:50:34 | INFO     | DF | Running on device cuda:0
2025-03-06 09:50:34 | INFO     | DF | Model loaded
✅ Enhanced Audio Saved at given output_file_path


Batch process on all files in a folder 

In [15]:
folder_path = r"C:\Users\User_1\Desktop\Comm-sounds"

In [16]:
audio_files = glob.glob(os.path.join(folder_path, '*.*'))
audio_extensions = ['.mp3', '.wav', '.flac', '.ogg', '.aac']  # Add more extensions if needed

audio_files = [f for f in audio_files if os.path.splitext(f)[1].lower() in audio_extensions]

for audio_file in audio_files:
    file_name = os.path.basename(audio_file)
    output_file_path = os.path.join(folder_path, f"enhanced_{os.path.splitext(file_name)[0]}.wav")
    
    denoising_deepfilternet(audio_file, output_file_path)

2025-03-06 09:58:14 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-06 09:58:14 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-06 09:58:14 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-06 09:58:14 | INFO     | DF | Found checkpoint C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3\checkpoints\model_120.ckpt.best with epoch 120
2025-03-06 09:58:14 | INFO     | DF | Running on device cuda:0
2025-03-06 09:58:14 | INFO     | DF | Model loaded
✅ Enhanced Audio Saved at given output_file_path
2025-03-06 09:58:15 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-06 09:58:15 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\User_1\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-06 09:58:15 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-06 09:58:15 | INFO     | DF | Found checkpoint C